# <span style="color:crimson">Data Wrangling</span>

### Aggregating and Restructuring the  Dataset

For this project, we have transaction-level dataset. In other words, each observation in the raw dataset is for a single transaction - one item, one customer, one purchase.

Therefore, to get the customer-level data (since we want to build clusters for customers, not transactions) we need to aggregate transactions by customer and engineer customer-level features.

## This is an example of Unsupervised Clustering

### Project Scope: Unsupervised Customers

Our client is an online retailer based in the United Kingdom (UK). They sell all occasion gifts and many of their customers are wholesellers. Most of their customers are bsed in the UK but they also have a small percentage of customers form other countries. They want to create groups of these international customers based on their previous purchase patterns. Their goal is to provide more tailored services and improve the way they market. to these international customers.

**Current Solution:** 
Currently the retailer simply groups their international customers by country. This is inefficient because,
1. There are large number of countries which defeats the purpose of creating groups.
2. Some countries have very few customers.
3. This approach treats the large and small customers the same regardlessof their purchase patterns.


**Our role:** 
The retailer has hired us to help them create 3 clusters of their international customers through a data driven approach. They have provided us with a data set with past purchased data at the transaction level. our task is to build a clustering model using that dataset. Our clustering model should factor in both
1. aggregate sales patterns, and
2. specific items purchased.

**Goal:**
Build a clustering model to create groups of international customers based on their previous purchase patterns.

### Specifics

**Machine Learning task:** Unsupervised Clustering

**Target variable:** N/A

**Input variables:** Refer to data dictionary file - "Data Dictionary - Unsupervised Customers"

**Deliverable:** Clusters for customers

**Success Criteria:** N/A

### This project will be carried out in 4 stages

**1. Data Wrangling:** Aggregating and Restructuring the dataset


**2. Dimensionality Reduction:** Breaking down the dimensions


**3. Principal Component Analysis (PCA):** Analyzing correlated features and explained variance


**4. Cluster Analysis:** Creating and analyzing clusters

## Importing the libraries

In [3]:
# NumPy for numerical computing
import numpy as np

# Pandas for DataFrames
import pandas as pd

# Matplotlib for visualization
from matplotlib import pyplot as plt

# display plots in the notebook
%matplotlib inline 

# Seaborn for easier visualization
import seaborn as sns
sns.set_style('darkgrid')

#### Read in the dataset. The file name is 'int_online_tx.cav'

In [5]:
df = pd.read_csv('int_online_tx.csv')

In [6]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,12/1/10 8:45,3.75,12583.0,France
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,12/1/10 8:45,3.75,12583.0,France
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,12/1/10 8:45,3.75,12583.0,France
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,12/1/10 8:45,0.85,12583.0,France
4,536370,21883,STARS GIFT TAPE,24,12/1/10 8:45,0.65,12583.0,France


In [21]:
#chekc the size of the data
df.shape

(35116, 8)

## I. Transaction-Level Data Cleaning

In [10]:
#check for missing values in each features
df.isnull().sum()

InvoiceNo         0
StockCode         0
Description       0
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID     1418
Country           0
dtype: int64

**Looks like 'CustomerID' has missing observations...** Now the question is should we label them as missing (as for categorical features) or should we flag and fill them (as for numeric features)?

In this case, we should do neither. Instead, we simply need to drop transactions with missing CustomerID.
we are trying to cluster customers in order to provide more tailored service. That means transactions with missing 'CustomerID' are actually pointless.

Even if we keep those missing CustomerID's and just label them as 'Missing'. When we roll up to the customer level, we'd essentially attribute all of those transactions to a single "Missing" customer.

In [11]:
#drop observations with null values. Only keep transactions with CustomerID's
clean_df = df[df.CustomerID.notnull()]

In [13]:
clean_df.shape

(33698, 8)

In [14]:
clean_df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [15]:
#check data types of all the features
clean_df.dtypes

InvoiceNo        int64
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [16]:
#As a good practive, let's convert customerID to integer type
clean_df['CustomerID'] = clean_df.CustomerID.astype(int)

/Users/swapnil/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
#verify
clean_df.dtypes

InvoiceNo        int64
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID       int64
Country         object
dtype: object

#### Finally, there's one feature we can create at the transaction level: 'Sales'

If we look at the current transaction-level dataset, there aren't any features that tell us how much money the customer spent on each transaction. Using <code style="color:steelblue">'Quantity'</code> and <code style="color:steelblue">'UnitPrice'</code>, we can derive a feature called <code style="color:steelblue">'Sales'</code>


**Let's create a <code style="color:steelblue">'Sales'</code> feature.**

In [1]:
# Create 'Sales' feature
clean_df['Sales'] = (clean_df.Quantity*clean_df.UnitPrice)

NameError: name 'clean_df' is not defined

In [19]:
# Display first 5 Sales values in the transaction dataset
clean_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,12/1/10 8:45,3.75,12583,France,90.0
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,12/1/10 8:45,3.75,12583,France,90.0
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,12/1/10 8:45,3.75,12583,France,45.0
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,12/1/10 8:45,0.85,12583,France,10.2
4,536370,21883,STARS GIFT TAPE,24,12/1/10 8:45,0.65,12583,France,15.6


## II. Customer-Level Feature Engineering

Now that we have a cleaned transaction-level dataset, it's time to roll it up to the customer level so that we have 1 customer per row.


We want 1 customer per row, and we want the features to represent information such as:
* Number of unique purchases by the customer<br>
* Average cart value for the customer<br>
* Total sales for the customer<br>



To do so, we'll use:<br>
1. <code style="color:steelblue">.groupby()</code> to roll up by customer.<br>
2. <code style="color:steelblue">.agg()</code> to engineer aggregated features.

#### First, we'll aggegrate invoice data by customer and engineer 1 feature:

<code style = "color:steelblue">'total_transactions'</code> - the total number of unique transactions for each customer.

In [22]:
# Aggegrate invoice data
a = clean_df.groupby('CustomerID').InvoiceNo.agg(['nunique'])

<code style="color:crimson">'nunique'</code> counts the number of unique values within each group.

In [23]:
# Rename the aggregate column
a.columns = ['total_transactions']

In [24]:
# Display total unique transactions first 5 customers
a.head()

,total_transactions
CustomerID,
12347,7
12348,4
12349,1
12350,1
12352,8
...,...
17444,3
17508,1
17828,4


#### Next, we'll aggregate product data by customer, and engineer 2 new features:

<code style="color:steelblue">'total_products'</code> - the total number of products purchased by each customer<br>
<code style="color:steelblue">'total_unique_products'</code> - the number of unique products purchased by each customer.


In [29]:
b = clean_df.groupby('CustomerID').StockCode.agg(['count','nunique'])

In [30]:
b.columns = ['total_products','total_unique_products']

In [31]:
b

,total_products,total_unique_products
CustomerID,,
12347,182,103
12348,31,22
12349,73,73
12350,17,17
12352,85,59
...,...,...
17444,135,132
17508,22,22
17828,67,54


**C.** Next, we'll aggregate sales data by customer and engineer 2 new features:

<code style="color:steelblue">'total_sales'</code> - the total sales for each customer.<br>
<code style="color:steelblue">'avg_product_value'</code> - the average value of the products purchased by the customer


In [32]:
c = clean_df.groupby('CustomerID').Sales.agg(['sum','mean'])

In [35]:
c.columns = ['total_sales', 'avg_product_value']

In [38]:
c

,total_sales,avg_product_value
CustomerID,,
12347,4310.00,23.681319
12348,1797.24,57.975484
12349,1757.55,24.076027
12350,334.40,19.670588
12352,2506.04,29.482824
...,...,...
17444,2940.04,21.778074
17508,387.31,17.605000
17828,1820.09,27.165522


#### We won't always be able to easily roll up to customer-level directly... Sometimes, it is easier to create intermediary levels first.


For example, say we wanted to calculate the average cart value for each customer. We would first aggregate cart data at the "cart-level."

We'll group by 'CustomerID' AND by 'InvoiceID'. Remember, we're treating each invoice as a "cart."
Then, we'll calculate 'cart_value' by taking the sum of the Sales column. This is the total sales by invoice (i.e. cart).


In [41]:
# Aggregate cart-level data (i.e. invoice-level)
cart = clean_df.groupby(['CustomerID' , 'InvoiceNo']).Sales.agg(['sum'])
cart.columns = ['cart_value']
cart

cart_value
CustomerID InvoiceNo            
12347      537626         711.79
           542237         475.39
           549222         636.25
           556201         382.52
           562032         584.91
...                          ...
17828      557668          30.00
           563936         213.50
           576897         743.43
17829      543911         889.24
17844      564428          51.56

[1536 rows x 1 columns]

**D.** Finally, we will aggregate cart data by customer, and engineer 3 new features:

<code style="color:steelblue">'avg_cart_value'</code> - average cart value by customer.

<code style="color:steelblue">'min_cart_value'</code> - minimum cart value by customer.

<code style="color:steelblue">'max_cart_value'</code> - maximum cart value by customer.


In [42]:
d = cart.groupby('CustomerID').cart_value.agg(['mean', 'min', 'max'])

In [43]:
d.columns = ['avg_cart_value','min_cart_value','max_cart_value']

In [44]:
d

,avg_cart_value,min_cart_value,max_cart_value
CustomerID,,,
12347,615.714286,224.82,1294.32
12348,449.310000,227.44,892.80
12349,1757.550000,1757.55,1757.55
12350,334.400000,334.40,334.40
12352,313.255000,120.33,840.30
...,...,...,...
17444,980.013333,550.94,1217.64
17508,387.310000,387.31,387.31
17828,455.022500,30.00,833.16


## <span style="color:RoyalBlue">Merging/Joining</span>

Great, now we have multiple dataframes that each contain customer-level features.
* <code style="color:crimson">a:</code> total_transactions
* <code style="color:crimson">b:</code>total_products and total_Unique_Products
* <code style="color:crimson">c:</code>total_sales, avg_product_value
* <code style="color:crimson">d:</code> avg_cart_value, min_cart_value, and max_cart_value

Next, all we need to do is **join / merge** them all together.

* Just pick one of the customer-level dataframes and join it to a list of the others.
* By default, it will join the dataframes on their <code style="color:steelblue">index</code>. In this case, it will join by <code style="color:steelblue">CustomerID.</code>

In [46]:
customer_df = a.join([b,c,d])
customer_df

,total_transactions,total_products,total_unique_products,total_sales,avg_product_value,avg_cart_value,min_cart_value,max_cart_value
CustomerID,,,,,,,,
12347,7,182,103,4310.00,23.681319,615.714286,224.82,1294.32
12348,4,31,22,1797.24,57.975484,449.310000,227.44,892.80
12349,1,73,73,1757.55,24.076027,1757.550000,1757.55,1757.55
12350,1,17,17,334.40,19.670588,334.400000,334.40,334.40
12352,8,85,59,2506.04,29.482824,313.255000,120.33,840.30
...,...,...,...,...,...,...,...,...
17444,3,135,132,2940.04,21.778074,980.013333,550.94,1217.64
17508,1,22,22,387.31,17.605000,387.310000,387.31,387.31
17828,4,67,54,1820.09,27.165522,455.022500,30.00,833.16


Finally, let's save <code style="color:crimson">customer_df</code> as our base table for further analysis.


* Save the file as <code style="color:crimson">'analytical_base_table.csv'</code>

In [143]:
customer_df.to_csv('analytical_base_table.csv')